In [ ]:
# Instalación (Colab/Local)
# Instalación (Colab/Local)
!pip install -U "langchain>=0.2.11" "langchain-core>=0.2.33" "langchain-community>=0.2.11" "langchain-openai>=0.2.1" "langsmith>=0.1.97"
# Opcionales para el assignment:
!pip install -U faiss-cpu chromadb tavily-python duckduckgo-search langchain-text-splitters

import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.9/410.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.47
    Uninstalling langsmith-0.4.47:
      Successfully uninstalled langsmith-0.4.47
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 require

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-nano", temperature=0)  # modelo sugerido

# Hola LLM
resp = llm.invoke("Definí 'Transformer' en una sola oración.")
print(resp.content)

Un Transformer es una arquitectura de redes neuronales basada en mecanismos de atención que procesa secuencias en paralelo, captura dependencias a largo plazo y utiliza codificación posicional para mantener el orden sin recurrencias.


In [3]:
from langchain_openai import ChatOpenAI

# Completar parámetros básicos (ver [1] y [12])
MODEL = "gpt-5-mini"         # p.ej., "gpt-5-mini"
TEMP = 0.0            # 0.0–1.0 (determinismo vs creatividad)

llm = ChatOpenAI(model=MODEL, temperature=TEMP)
print(llm.invoke("Hola! Decime tu versión en una línea.").content)

Soy ChatGPT, basado en el modelo GPT-4 con conocimiento hasta junio de 2024.


In [4]:
prompts = [
    "Escribí un tuit (<=20 palabras) celebrando un paper de IA.",
    "Dame 3 bullets concisos sobre ventajas de los Transformers."
]

for t in [0.0, 0.5, 0.9]:
    llm_t = ChatOpenAI(model=MODEL, temperature=t)
    outs = [llm_t.invoke(p).content for p in prompts]
    print(f"\n--- Temperature={t} ---")
    for i, o in enumerate(outs, 1):
        print(f"[{i}] {o}")


--- Temperature=0.0 ---
[1] ¡Felicidades equipo! Nuevo paper de IA que redefine posibilidades: innovación brillante, impacto real. Leyéndolo ya. #IA #Ciencia
[2] - Procesamiento paralelo: la atención permite calcular representaciones de todos los tokens simultáneamente, acelerando el entrenamiento frente a RNNs.  
- Captura dependencias a largo plazo: la atención conecta directamente cualquier par de posiciones, mejorando el modelado de contextos extensos.  
- Escalabilidad y transferencia: funcionan bien al aumentar parámetros/datos y facilitan preentrenamiento y fine‑tuning en múltiples tareas.

--- Temperature=0.5 ---
[1] ¡Bravo al equipo! Paper de IA que abre nuevas fronteras: innovación, rigor y futuro para todos.
[2] - Permiten procesamiento altamente paralelo (self-attention), acelerando el entrenamiento frente a RNNs.  
- Capturan dependencias a largo plazo de forma directa y flexible, modelando relaciones entre cualquier par de tokens.  
- Escalan bien y facilitan el preentre

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, temperature=TEMP, top_p=1)
print(llm.invoke("Escribí un haiku sobre evaluación de modelos.").content)

Métricas hablan  
miden verdad y fallas  
la curva habla


In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Sos un asistente conciso, exacto y profesional."),
    ("human",  "Explicá {tema} en <= 3 oraciones, con un ejemplo real.")
])

chain = prompt | llm  # LCEL: prompt → LLM
print(chain.invoke({"tema": "atención multi-cabeza"}).content)

La atención multi-cabeza divide consultas, claves y valores en varias sub-proyecciones que calculan atenciones paralelas independientes y luego concatenan sus salidas, permitiendo combinar diferentes relaciones entre tokens. Cada cabeza puede especializarse (alineamiento léxico, dependencia sintáctica, relaciones de largo alcance), aumentando la capacidad del modelo para representar información compleja. Por ejemplo, al traducir "The cat that chased the mouse sat on the mat", una cabeza puede alinear "cat"→"gato" mientras otra captura la dependencia larga entre "cat" y el verbo "sat" para asegurar la concordancia.


In [7]:
from typing import List
from pydantic import BaseModel

class Resumen(BaseModel):
    title: str
    bullets: List[str]

llm_json = llm.with_structured_output(Resumen)  # garantiza JSON válido que cumple el esquema

pedido = "Resumí en 3 bullets los riesgos de la 'prompt injection' en LLM apps."
res = llm_json.invoke(pedido)


In [8]:
res.model_dump_json()

'{"title":"Riesgos de \'prompt injection\' en LLM apps","bullets":["Exfiltración de datos sensibles: un atacante puede inducir al modelo a revelar secretos o información confidencial presente en el contexto o prompts previos.","Manipulación del comportamiento y seguridad: instrucciones maliciosas pueden anular salvaguardas y provocar respuestas inseguras, desinformación o acciones no deseadas.","Compromiso de integridad y confianza del sistema: permite eludir controles, ejecutar acciones no autorizadas o afectar componentes downstream (APIs, código), minando la fiabilidad y seguridad de la aplicación."]}'

In [9]:
# Esqueleto sugerido para 1) y 2)
from pydantic import BaseModel

class Traduccion(BaseModel):
    text: str
    lang: str

traductor = llm.with_structured_output(Traduccion)
salida = traductor.invoke("Traducí al portugués: 'Excelente trabajo del equipo'.")
print(salida)

# Q&A con contexto (sin RAG)
from langchain_core.prompts import ChatPromptTemplate
QA_prompt = ChatPromptTemplate.from_messages([
    ("system", "Respondé SOLO usando el contexto. Si no alcanza, decí 'No suficiente contexto'."),
    ("human",  "Contexto:\n{contexto}\n\nPregunta: {pregunta}\nRespuesta breve:")
])
salida = (QA_prompt | llm).invoke({
    "contexto": "OpenAI y LangChain permiten structured output con JSON...",
    "pregunta": "¿Qué ventaja tiene structured output?"
})
print(salida)

text='Excelente trabalho da equipe' lang='pt'
content='No suficiente contexto' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 204, 'prompt_tokens': 54, 'total_tokens': 258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-ChhoD9pcnmpvpY3etdkYgb8adduOj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--9d54a318-4691-47c0-9455-7f7b675cdfd8-0' usage_metadata={'input_tokens': 54, 'output_tokens': 204, 'total_tokens': 258, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}}


In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, temperature=TEMP)

# Zero-shot
zs_prompt = ChatPromptTemplate.from_messages([
    ("system", "Sos un asistente conciso y exacto."),
    ("human",  "Clasificá el sentimiento de este texto como POS, NEG o NEU:\n\n{texto}")
])

# Few-shot (1–2 ejemplos)
fs_prompt = ChatPromptTemplate.from_messages([
    ("system", "Sos un asistente conciso y exacto."),
    ("human",  "Ejemplo:\nTexto: 'El producto superó mis expectativas'\nEtiqueta: POS"),
    ("human",  "Ejemplo:\nTexto: 'La entrega fue tarde y vino roto'\nEtiqueta: NEG"),
    ("human",  "Texto: {texto}\nEtiqueta:")
])

textos = [
    "Me encantó la experiencia, repetiría.",
    "No cumple lo prometido; decepcionante.",
    "Está bien, nada extraordinario."
]

print("== Zero-shot ==")
for t in textos:
    print((zs_prompt | llm).invoke({"texto": t}).content)

print("\n== Few-shot ==")
for t in textos:
    print((fs_prompt | llm).invoke({"texto": t}).content)

== Zero-shot ==
POS
NEG
NEU

== Few-shot ==
Etiqueta: POS
NEG
Etiqueta: NEG

Justificación breve: Expresa satisfacción tibia ("está bien") pero sin entusiasmo ("nada extraordinario"), por lo que no se considera claramente positiva.


In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


long_text = """El origen del juego ajedrez sigue siendo un misterio, pero la versión más aceptada sugiere que el ajedrez fue inventado en Asia, probablemente en India (Imperio Gupta), con el nombre de chaturanga, y desde ahí se extendió a China, Rusia, Persia y Europa, donde se estableció la normativa vigente. Sin embargo, investigaciones recientes indican un posible origen chino, en la región entre Uzbekistán y la antigua Persia, que se podría remontar hasta el siglo III a. C.

Uno de los registros literarios más antiguos sobre el ajedrez es el poema persa Kar-Namag i Ardashir i Pabagan, escrito en el siglo VI. A partir de esta era su evolución está mejor documentada y ampliamente aceptada en el mundo académico. Tras la conquista de Persia por los árabes, éstos asimilaron el juego y lo difundieron en Occidente, llevándolo al norte de África y Europa, e incluso la actual España e Italia alrededor del siglo X, desde donde se extendió al resto del continente llegando a la región de Escandinavia e Islandia. En Oriente, el ajedrez se ha expandido desde su versión china, el Xiangqi, a Corea y Japón en el siglo X.

En el siglo XV el juego fue ampliamente difundido en Europa y entre las variantes existentes del juego, la europea fue la que se destacó por la velocidad indicada y además por la inclusión de la dama y el alfil. A pesar de que en esa época ya existía literatura de ajedrez, fue en este período cuando comenzaron a surgir los primeros análisis de aperturas debido a las nuevas posibilidades de juego.

Las partidas comenzaron a ser registradas con mayor frecuencia y se publicaron más estudios teóricos. En el siglo XVIII se fundaron los primeros clubes para la práctica de ajedrez y federaciones deportivas en Europa, y debido a la gran cantidad de pequeños torneos que ocurren por todo el continente, en 1851 se celebró el primer torneo internacional en Londres. La popularidad de las competiciones internacionales llevó a la creación del título de campeón del mundo, ganado por Wilhelm Steinitz en 1886, y, en 1924 se fundó la Federación Internacional de Ajedrez (FIDE), en París, que organiza la primera Olimpiada de Ajedrez y el mundial femenino, ganado por Vera Menchik.

A finales de la década de 1950, con la popularización de las computadoras, comenzaron a surgir los primeros programas que juegan al ajedrez, que acompañaban la evolución del tratamiento de la información e introducirán el juego en la era moderna con competiciones en línea y facilitando el análisis de las partidas.

Origen
El origen del ajedrez sigue siendo una cuestión de debate entre los historiadores del ajedrez,[1]​[2]​[3]​ sin embargo, la teoría más difundida[4]​ es que fue creado en la India durante el Imperio Gupta alrededor del siglo VI.[5]​ Esta teoría se confirma por los primeros registros literarios persas y por el análisis de la etimología de las palabras utilizadas en el juego y su co-evolución con el ajedrez.[6]​

Sin embargo, las teorías alternativas proponen que el ajedrez fue creado durante un período anterior, en diferentes lugares, como China,[7]​ Irán[8]​ y Afganistán.[9]​ Estas versiones exploran evidencias arqueológicas, militares[10]​ y literarias de la evidencia filogenética[11]​ para desafiar la teoría de la India. Las similitudes entre el chaturanga y el xiangqi, considerado la versión china de ajedrez, son exploradas indicando que estos juegos podrían haberse influido entre sí por medio del contacto entre las civilizaciones a través de la ruta de la seda, asimilando algunos aspectos de sus reglas y formando versiones híbridas,[12]​ que podrían remontarse a la antigua Grecia y la conquista de Alejandro Magno sobre Asia Menor en el siglo III antes de Cristo.[13]​ Existe la perspectiva de que, en el futuro, nuevos análisis de la literatura existente y el descubrimiento de más objetos arqueológicos en la India y China permitan esclarecer definitivamente el origen del ajedrez.[3]​

India

Krishna y Radha jugando al chaturanga en un tablero del Ashtāpada.
Según Harold Murray, el análisis filológico conecta el juego con claridad a la palabra chaturanga,[14]​ que designaba a las cuatro partes del ejército indio —carros, elefantes, caballería e infantería— desde el siglo V a. C.[15]​ En un principio, el partido se jugó en el consejo de Ashtāpada, otro juego cuya significación se estableció alrededor del siglo V a. C., y sugirió un objeto familiar.[16]​

El chaturanga es considerado el juego más antiguo con características esenciales de la definición del juego encontradas en las versiones posteriores — dos jugadores se enfrentan en un acuerdo inicial y simétrico de las piezas, con piezas de movimientos diferentes y la victoria depende de la captura de una única pieza.[17]​ No está claro si el chaturanga utilizaba dados para designar a sus movimientos, aunque la gran mayoría de los juegos indios los utilizasen.[18]​"""

# Split en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks = splitter.split_text(long_text)

# Cadena para resumir un chunk
chunk_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "Resumí el siguiente fragmento en 2–3 bullets, claros y factuales."),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
chunk_summary = chunk_summary_prompt | llm

bullets = [chunk_summary.invoke({"input": c}).content for c in chunks]

# Reduce (combinar resultados)
reduce_prompt = ChatPromptTemplate.from_messages([
    ("system", "Consolidá bullets redundantes y producí un resumen único y breve."),
    ("human", "Bullets:\n{bullets}\n\nResumen final (<=120 tokens):")
])

final = (reduce_prompt | llm).invoke({"bullets": "\n".join(bullets)}).content
print(final)

El origen del ajedrez es incierto: la hipótesis mayoritaria lo sitúa en la India (chaturanga, Imperio Gupta, s. VI), aunque hay teorías que lo atribuyen a China, Persia o Asia central y fechas más antiguas. Desde Persia pasó a los árabes y de ahí a Europa (península Ibérica) y al Este asiático. En Europa (s. XV) cambiaron dama y alfil; en los siglos XIX–XX se consolidaron torneos, el título mundial (1886) y la FIDE (1924). Desde los años 50 la informática modernizó el análisis y el juego en línea.


In [12]:
from typing import List, Optional
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

class Entidad(BaseModel):
    tipo: str   # p.ej., 'ORG', 'PER', 'LOC'
    valor: str

class ExtractInfo(BaseModel):
    titulo: Optional[str]
    fecha: Optional[str]
    entidades: List[Entidad]

extractor = llm.with_structured_output(ExtractInfo)
texto = "OpenAI anunció una colaboración con la Universidad Catolica del Uruguay en Montevideo el 05/11/2025."
extractor.invoke(f"Extraé titulo, fecha y entidades (ORG/PER/LOC) del siguiente texto:\n\n{texto}")

ExtractInfo(titulo='OpenAI anunció una colaboración con la Universidad Catolica del Uruguay en Montevideo', fecha='05/11/2025', entidades=[Entidad(tipo='ORG', valor='OpenAI'), Entidad(tipo='ORG', valor='Universidad Catolica del Uruguay'), Entidad(tipo='LOC', valor='Montevideo')])

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS  # Si usás Chroma, importá su VectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.documents import Document

docs_raw = [
    "LangChain soporta structured output con Pydantic.",
    "RAG combina recuperación + generación para mejor grounding.",
    "OpenAIEmbeddings facilita embeddings para indexar textos."
]
docs = [Document(page_content=t) for t in docs_raw]

# Split y vector store
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

emb = OpenAIEmbeddings()
vs = FAISS.from_documents(chunks, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 4})

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Respondé SOLO con el contexto. Si no alcanza, decí 'No suficiente contexto'."),
    ("human",  "Contexto:\n{context}\n\nPregunta: {input}")
])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

rag_chain.invoke({"input": "¿Qué ventaja clave aporta RAG?"})

{'input': '¿Qué ventaja clave aporta RAG?',
 'context': [Document(id='8f76bf75-d6c4-4b70-80a4-d141ddff8fab', metadata={}, page_content='RAG combina recuperación + generación para mejor grounding.'),
  Document(id='c1910cdf-4374-4e8a-adb0-7fd04be5c924', metadata={}, page_content='OpenAIEmbeddings facilita embeddings para indexar textos.'),
  Document(id='d12257d8-dd8c-4c5c-a203-1d02112fcd1a', metadata={}, page_content='LangChain soporta structured output con Pydantic.')],
 'answer': 'RAG combina recuperación + generación para mejor grounding.'}